In [60]:
import pandas as pd
from textblob import TextBlob

In [36]:
stock_prices = pd.read_csv('/content/GOOG_historical_data.csv')
news = pd.read_csv('/content/raw_analyst_ratings.csv')

In [52]:
stock_prices = stock_prices.drop(columns=['Dividends','Stock Splits'])

In [53]:
stock_prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,2.490664,2.591785,2.390042,2.499133,2.496292,897427216
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.694573,458857488
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.721690,366857939
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.608991,306396159
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.637103,184645512


## **Date Alignment**

In [54]:
print("News dataset dates:")
print(news['date'].head())

print("\nStock prices dataset dates:")
print(stock_prices['Date'].head())

News dataset dates:
0    2020-06-05
1    2020-06-03
2    2020-05-26
3    2020-05-22
4    2020-05-22
Name: date, dtype: object

Stock prices dataset dates:
0    2004-08-19
1    2004-08-20
2    2004-08-23
3    2004-08-24
4    2004-08-25
Name: Date, dtype: object


To make sure that the dates in the news and stock price datasets are aligned, we must first normalize the formats. If the two datasets have different date formats (e.g., one includes timestamps and timezones while the other does not)
- Converting all date columns to a common format using pd.to_datetime.

- Extracting only the date component (removing time and timezone information)

In [55]:
# Convert to datetime
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
news['date'] = pd.to_datetime(news['date'], format='mixed')

In [56]:
news_df = pd.DataFrame(news)
stock_df = pd.DataFrame(stock_prices)

In [57]:
#Removes timezone information from the datetime values
news['date'] = pd.to_datetime(news['date'].astype(str)).dt.tz_localize(None).dt.date

# Normalize the 'Date' column in the stock prices dataset
stock_prices['Date'] = pd.to_datetime(stock_prices['Date']).dt.date

# Display results to confirm
news[['date']].head(), stock_prices[['Date']].head()

(         date
 0  2020-06-05
 1  2020-06-03
 2  2020-05-26
 3  2020-05-22
 4  2020-05-22,
          Date
 0  2004-08-19
 1  2004-08-20
 2  2004-08-23
 3  2004-08-24
 4  2004-08-25)

In [71]:
aligned_df = pd.merge(news, stock_prices, left_on='date', right_on='Date', how='inner')
aligned_df.head()

,Unnamed: 0,headline,url,publisher,date,stock,sentiment,sentiment_class,Date,Open,High,Low,Close,Adj Close,Volume
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,0.0,Neutral,2020-06-05,70.658501,72.252502,70.300003,71.919502,71.837753,34698000
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,0.0,Neutral,2020-06-03,71.915001,72.327599,71.488853,71.819000,71.737366,25124000
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,0.0,Neutral,2020-05-26,71.863503,72.050003,70.606499,70.850998,70.770462,41212000
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,0.0,Neutral,2020-05-22,69.835503,70.638000,69.591499,70.521004,70.440842,26188000
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,0.0,Neutral,2020-05-22,69.835503,70.638000,69.591499,70.521004,70.440842,26188000


## **Sentiment Analysis**

In [61]:
news['sentiment'] = news['headline'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [62]:
news.head()

,Unnamed: 0,headline,url,publisher,date,stock,sentiment
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,0.0
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,0.0
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,0.0
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,0.0
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,0.0


In [66]:
def classify_sentiment(polarity):
    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [65]:
news['sentiment_class'] = news['sentiment'].apply(classify_sentiment)

# Display the result
news[['headline', 'sentiment', 'sentiment_class']].head()

,headline,sentiment,sentiment_class
0,Stocks That Hit 52-Week Highs On Friday,0.0,Neutral
1,Stocks That Hit 52-Week Highs On Wednesday,0.0,Neutral
2,71 Biggest Movers From Friday,0.0,Neutral
3,46 Stocks Moving In Friday's Mid-Day Session,0.0,Neutral
4,B of A Securities Maintains Neutral on Agilent...,0.0,Neutral


The **polarity** value is a float within the range of [-1, 1]. Positive values indicate positive sentiment, negative values indicate negative sentiment, and values close to 0 indicate neutral sentiment.

-----

In [68]:
len(stock_prices)

5020

In [69]:
len(news)

1407328

In [74]:
len(aligned_df)

55230

In [78]:
news_date_counts = news.groupby('date').size()
multiple_articles_dates = news_date_counts[news_date_counts > 1]
print(multiple_articles_dates)

date
2011-04-28      2
2011-04-29      2
2011-05-02      9
2011-05-03      3
2011-05-05      3
             ... 
2020-06-07     25
2020-06-08    765
2020-06-09    804
2020-06-10    806
2020-06-11    544
Length: 2218, dtype: int64


In [79]:
aligned_date_counts = aligned_df.groupby('date').size()
multiple_articles_dates = aligned_date_counts[aligned_date_counts > 1]
print(multiple_articles_dates)

date
2011-04-28      2
2011-04-29      2
2011-05-02      9
2011-05-03      3
2011-05-05      3
             ... 
2020-06-05    932
2020-06-08    765
2020-06-09    804
2020-06-10    806
2020-06-11    544
Length: 2091, dtype: int64


----

## **Analysis**

To calculate the daily stock returns, which represent the percentage change in daily closing prices, can use the pct_change() function from pandas. This function calculates the percentage difference between the current and previous values in a Close column

In [80]:
stock_prices['Daily_Return'] = stock_prices['Close'].pct_change() * 100
print(stock_prices[['Date', 'Close', 'Daily_Return']].head())

         Date     Close  Daily_Return
0  2004-08-19  2.499133           NaN
1  2004-08-20  2.697639      7.942989
2  2004-08-23  2.724787      1.006362
3  2004-08-24  2.611960     -4.140766
4  2004-08-25  2.640104      1.077510


- 2004-08-20: The stock's closing price is 2.697639. The daily return is 7.94%. This means that, compared to the previous day, the stock's price increased by 7.94% on 2004-08-20.

- 2004-08-23: The stock's closing price is 2.724787. The daily return is 1.01%. This means the stock price increased by 1.01% compared to 2004-08-20.

